In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.train.estimator import Estimator
import json

In [2]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [3]:
# load experiment cfg
with open("experiment_cfg.json", "r") as cfg_file:
    cfg = json.load(cfg_file)

## Training Script

In [39]:
%%writefile ./src/train.py

from azureml.core import Run
import os
import joblib
from argparse import ArgumentParser
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import recall_score, precision_score, hamming_loss, zero_one_loss, mean_absolute_error, mean_squared_error, r2_score

run = Run.get_context()

parser = ArgumentParser()
parser.add_argument('--input', dest='preprocessed_data')
args = parser.parse_args()

############################################################

# load data
if args.preprocessed_data:
    train_data = pd.read_csv(args.preprocessed_data + '/train_data.csv', sep=';', header=0)
    test_data = pd.read_csv(args.preprocessed_data + '/test_data.csv', sep=';', header=0)
else:
    train_data = run.input_datasets['train_data'].to_pandas_dataframe()
    test_data = run.input_datasets['test_data'].to_pandas_dataframe()
    
############################################################

# split train/test and feat/target for classifier
X_train = train_data[[ col for col in train_data.columns if col.startswith('feat')]]
y_train = train_data[[ col for col in train_data.columns if col.startswith('target')]].drop(['target_0'], axis=1)
X_test = test_data[[col for col in test_data.columns if col.startswith('feat')]]
y_test = test_data[[ col for col in test_data.columns if col.startswith('target')]].drop(['target_0'], axis=1)

############################################################

# train classifier
model = MultiOutputClassifier(SGDClassifier())
model.fit(X_train, y_train)

############################################################

# evaluate test data
y_pred = model.predict(X_test).round()
run.log_table(
    'test_evaluation_classification',
    {
        'precision_macro': [precision_score(y_test, y_pred, average='macro')],
        'precision_samples': [precision_score(y_test, y_pred, average='samples')],
        'recall_macro': [recall_score(y_test, y_pred, average='macro')],
        'recall_samples': [recall_score(y_test, y_pred, average='samples')],
        'hamming_loss': [hamming_loss(y_test, y_pred)],
        'zero_one_loss': [zero_one_loss(y_test, y_pred)]
    }
)

# evaluate train data
y_pred = model.predict(X_train).round()
run.log_table(
    'train_evaluation_classification',
    {
        'precision_macro_train': [precision_score(y_train, y_pred, average='macro')],
        'precision_samples_train': [precision_score(y_train, y_pred, average='samples')],
        'recall_macro_train': [recall_score(y_train, y_pred, average='macro')],
        'recall_samples_train': [recall_score(y_train, y_pred, average='samples')],
        'hamming_loss_train': [hamming_loss(y_train, y_pred)],
        'zero_one_loss_train': [zero_one_loss(y_train, y_pred)]
    }
)

############################################################

# save model
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/model.pkl')

############################################################

# split train/test and feat/target for regressor
X_train = train_data[[ col for col in train_data.columns if col.startswith('feat')]]
y_train = train_data[[ col for col in train_data.columns if col.startswith('target')]][['target_0']]
X_test = test_data[[col for col in test_data.columns if col.startswith('feat')]]
y_test = test_data[[ col for col in test_data.columns if col.startswith('target')]][['target_0']]
y_test = y_test.values[:,0]

############################################################

# train regressor
model_regressor = SGDRegressor()
model_regressor.fit(X_train, y_train)

############################################################

# evaluate test data
y_pred = model_regressor.predict(X_test)
run.log_table(
    'test_evaluation_regression',
    {
        'mae': [mean_absolute_error(y_test, y_pred)],
        'mse': [mean_squared_error(y_test, y_pred)],
        'r2': [r2_score(y_test, y_pred)]
    }
)

# evaluate train data
y_pred = model_regressor.predict(X_train)
run.log_table(
    'train_evaluation_regression',
    {
        'mae_train': [mean_absolute_error(y_train, y_pred)],
        'mse_train': [mean_squared_error(y_train, y_pred)],
        'r2_train': [r2_score(y_train, y_train)]
    }
)

############################################################

# save regressor model
joblib.dump(value=model_regressor, filename='outputs/model_regressor.pkl')

run.complete()

Overwriting ./src/train.py


## Create Estimator

In [40]:
est = Estimator(entry_script='train.py', source_directory='src',
                inputs=[ws.datasets[cfg['train_dataset']].as_named_input('train_data'), 
                        ws.datasets[cfg['test_dataset']].as_named_input('test_data')   ],
                compute_target='local', environment_definition=ws.environments[cfg['env_name']])

## Run Experiment

In [41]:
exp = Experiment(ws, cfg['experiment_name'])
run = exp.submit(est)
run.wait_for_completion(show_output=True)

RunId: SKLearnPrediction_1592489381_8dd05ecc
Web View: https://ml.azure.com/experiments/SKLearnPrediction/runs/SKLearnPrediction_1592489381_8dd05ecc?wsid=/subscriptions/793146d9-d4dc-4a73-9728-76c4ffd0cc0d/resourcegroups/rg_dynamics_test/workspaces/resdynml1test

Streaming azureml-logs/70_driver_log.txt

Entering context manager injector. Current time:2020-06-18T14:09:42.710140
Starting the daemon thread to refresh tokens in background for process with pid = 111472
Entering Run History Context Manager.
Preparing to call script [ train.py ] with arguments: []
After variable expansion, calling script [ train.py ] with arguments: []

2020-06-18 14:09:50.899423: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-06-18 14:09:50.899513: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_

/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_

/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_

/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_

/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_

/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_

/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_

/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/azureuser/.azureml/envs/azureml_8ae12e84272f7dd3cd3f437330398910/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for 

{'runId': 'SKLearnPrediction_1592489381_8dd05ecc',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-06-18T14:09:42.49862Z',
 'endTimeUtc': '2020-06-18T14:13:05.900662Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'b53c3175-1523-4d38-be17-363f9c78a592'},
 'inputDatasets': [{'dataset': {'id': 'ac4116cb-a165-49f4-b689-ff5b80bf4586'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'test_data', 'mechanism': 'Direct'}}, {'dataset': {'id': '8726bdba-1926-4ffc-aa05-4f928053208f'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'train_data', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {'test_data': {'dataLocation': {'dataset': {'id': 'ac4116cb-a165-49f4-b689-ff5b80bf4586',
      'name': None,
      'version': None},


## Register Models

In [ ]:
# register trained model
run.register_model(cfg['TrainedClassifier'], 'outputs/model.pkl')
run.register_model(cfg['TrainedRegressor'], 'outputs/model_regressor.pkl')